#Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install sentencepiece
!pip install transformers
!pip install googletrans==4.0.0rc1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd 
import numpy as np 
import json
import time
from google.colab import drive
import random
import regex as re
import matplotlib.pyplot as plt
import os
import sentencepiece

import torch
import torch.nn as nn

from transformers import AutoTokenizer
from transformers import AdamW
from transformers import XLMRobertaConfig, XLMRobertaModel, XLMRobertaTokenizer

from sklearn.model_selection import train_test_split

from torch.utils.data import TensorDataset, DataLoader

import googletrans
from googletrans import Translator
import logging

In [4]:
FOLDERNAME = '/content/drive/MyDrive/CS263/Final Project/Extracted_data'
%cd drive/My\ Drive
%cd $FOLDERNAME

/content/drive/My Drive
/content/drive/MyDrive/CS263/Final Project/Extracted_data


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim


In [6]:
train_data = pd.read_csv("train_df.csv")
dev_data = pd.read_csv("evaluation_df.csv")

# Drop rows with NaN values in 'text1' or 'text2'
# train = train_data.dropna()
# dev = dev_data.dropna()

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
# Delete columns that contain NaN sentences.
processed_data = train_data[train_data['text1'].notna()]
processed_data = processed_data[processed_data['text2'].notna()]

print("After removing NA text columns, we lose {0} rows.".format(train_data.shape[0] - processed_data.shape[0]))

After removing NA text columns, we lose 113 rows.


In [9]:
processed_data = processed_data.head(500)

In [10]:
# split into train and development.
train, dev = train_test_split(processed_data, test_size=0.1, random_state = 42)

In [11]:
train.head(2)

,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,Entities,Time,Narrative,Overall,Style,Tone,text1,text2
72,en,en,1498601317_1483894995,https://plainsman.com/article/2019-big-bo-classic,http://www.nigerianeye.com/2020/01/2020mfm-go-...,https://web.archive.org/web/plainsman.com/arti...,https://web.archive.org/web/www.nigerianeye.co...,4.0,4.000000,2.333333,4.000000,4.0,3.666667,2.666667,"Thursday, June 2, 2022 \nAlready a subscriber?...","Signup for FREE news updates, latest informati..."
182,en,en,1484452449_1483829079,https://timesofindia.indiatimes.com/city/luckn...,https://in-cyprus.com/weather-forecast-rain-an...,https://web.archive.org/web/timesofindia.india...,https://web.archive.org/web/in-cyprus.com/weat...,4.0,2.333333,1.000000,1.666667,2.0,1.000000,1.333333,Unnamed: 182,Today there will be isolated showers island-wi...


In [12]:
# class SiameseNetwork(nn.Module):
#     def __init__(self, bert_model):
#         super(SiameseNetwork, self).__init__()
#         self.bert_model = bert_model
#         # self.fc = nn.Linear(bert_model.config.hidden_size, 1)

#     def forward(self, text_inputs, attention_mask):
#         output = self.bert_model(text_inputs, attention_mask=attention_mask)
#         # pooled_output = outputs.pooler_output
#         # output = self.fc(pooled_output)
#         return output



#XLM-BERT

In [13]:
max_len = 512
batch_size = 5
lr = 5e-6
weight_decay = 1e-4
num_epochs = 40

In [14]:

device = torch.device("cuda")
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

#set_seed(self.config.seed)

In [15]:
def get_data_loader(data, batch_size_flg = True):
  tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
  input_ids, attention_masks, labels = [], [], []
  for idx, row in data.iterrows():
      text1, text2 = row['text1'], row['text2']
      encode_dict = tokenizer(text1,text2,
                                  max_length=max_len,
                                  padding='max_length',
                                  truncation=True,
                                  add_special_tokens=True
                                  )
      
      input_ids.append(encode_dict['input_ids'])
      attention_masks.append(encode_dict['attention_mask'])
      # Convert to only 1 label.
      labels.append([float(x) for x in [row['Geography'],row['Entities'],row['Time'],row['Narrative'],row['Overall'],row['Style'],row['Tone']]])

  input_ids = torch.tensor(input_ids)
  attention_masks = torch.tensor(attention_masks)
  labels = torch.tensor(labels)

  data = TensorDataset(input_ids, attention_masks, labels)
  if(batch_size_flg):
      data_loader = DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=True)
  else:
      data_loader = DataLoader(data)
  return data_loader

In [16]:
train_data_loader = get_data_loader(train)
eval_data_loader = get_data_loader(dev, False)

In [17]:
class MMRegressor(nn.Module):

    def __init__(self, model, hidden_size):
        
        super(MMRegressor, self).__init__()
        #self.config = XLMRobertaConfig.from_pretrained()
        self.reg_model = model

        self.fc1 = nn.Linear(hidden_size, 512)
        self.fc2 = nn.Linear(512, 7)
        self.activation = nn.GELU()

    def forward(self, input_ids, attention_mask):

        output1 = self.reg_model(input_ids, attention_mask)[1]
        logits1 = self.fc2(self.activation(self.fc1(output1)))

        output2 = self.reg_model(input_ids, attention_mask)[1]
        logits2 = self.fc2(self.activation(self.fc1(output2)))
        
        return logits1, logits2

In [18]:
def predict(model, data_loader):

    model.eval()
    test_pred, test_true = [], []
    with torch.no_grad():
        for idx, (ids, att, y) in enumerate(data_loader):
            y_pred = model(ids.to(device), att.to(device))
            y_pred = torch.squeeze(torch.add(torch.mul(y_pred[0], 0.5), torch.mul(y_pred[1], 0.5))).detach().cpu().numpy().tolist()
            y = y.squeeze().cpu().numpy().tolist()

            test_true.append(y[4])
            test_pred.append(y_pred[4])

        return test_true, test_pred


def calculate_weighted_loss( y_pred, y, criterion, loss_weights):
  loss = 0.0
  for i in range(7):
    y_pred_i, y_i = y_pred[:, i], y[:, i]
    loss += criterion(y_pred_i, y_i) * loss_weights[i]
  return loss

In [19]:
def train(model, model_path, train_loader, valid_loader, optimizer, epoches, loss_weights):
    logging.basicConfig(level=logging.INFO)
    best_pearson = 0.0
    criterion = nn.MSELoss()
    model.train()

    for i in range(epoches):
        start_time = time.time()
        train_loss_sum = 0.0
        
        print(f"—————————————————————— Epoch {i+1} ——————————————————————")
        #print(logging.info)
        
        for idx, (ids, att, y) in enumerate(train_loader):

            ids, att, y = ids.to(device), att.to(device), y.to(device)
            optimizer.zero_grad()
            y_pred1, y_pred2 = model(ids, att)
            y_pred1, y_pred2, y = torch.squeeze(y_pred1), torch.squeeze(y_pred2), torch.squeeze(y)

            loss1 = calculate_weighted_loss(y_pred1, y, criterion, loss_weights) #* self.config.losses_weights['forward_weight']
            loss2 = calculate_weighted_loss(y_pred2, y, criterion, loss_weights) #* self.config.losses_weights['forward_weight']
            loss_r = calculate_weighted_loss(y_pred1, y_pred2, criterion, loss_weights) #* self.config.losses_weights['rdrop_weight']     
            loss = (loss1 + loss2 + loss_r) /3

            
            loss.backward()
            optimizer.step()
            #schedule.step()
            train_loss_sum += loss.item()

            if (idx+1) % (len(train_loader) // 10) == 0:
                print("Epoch {:02d} | Step {:03d}/{:03d} | Loss {:.4f} | Time {:.2f}".format(i+1, idx+1, len(train_loader), train_loss_sum/(idx+1), time.time()-start_time))
        

        print("Start evaluating!")
        dev_true, dev_pred = predict(model, valid_loader)
        cur_pearson = np.corrcoef(dev_true, dev_pred)[0][1]
        
        if cur_pearson > best_pearson:
            best_pearson = cur_pearson
            torch.save(model.state_dict(), model_path)
        
        print("Current dev pearson is {:.4f}, best pearson is {:.4f}".format(cur_pearson, best_pearson))
        print("Time costed : {}s \n".format(round(time.time() - start_time, 3)))
        #print('logging',logging.info)

## Trainning

In [20]:
# Run model finetuning and save fine-tuned model.
# pre_trained_model = XLMRobertaModel.from_pretrained("xlm-roberta-large")
torch.cuda.empty_cache()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

pre_trained_model = XLMRobertaModel.from_pretrained("xlm-roberta-base")
config = XLMRobertaConfig.from_pretrained("xlm-roberta-base")
hidden_size = config.hidden_size
# hidden_size = 768

loss_weights = [0.5 if i == 4 else (1-0.5)/6 for i in range(7)]

model_name = f'XLM_Roberta_base.pth'
model_path = f"/content/drive/MyDrive/CS263/Final Project/Models/{model_name}"

model = MMRegressor(pre_trained_model, hidden_size)
model.to(device)

print(f"Model name for this run: {model_name}")

optimizer = AdamW(model.parameters(), lr=lr, weight_decay = weight_decay)
train(model, model_path, train_data_loader, eval_data_loader, optimizer, num_epochs, loss_weights)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model name for this run: XLM_Roberta_base.pth
—————————————————————— Epoch 1 ——————————————————————


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 01 | Step 009/090 | Loss 5.1406 | Time 3.23
Epoch 01 | Step 018/090 | Loss 4.9923 | Time 5.49
Epoch 01 | Step 027/090 | Loss 4.9280 | Time 7.75
Epoch 01 | Step 036/090 | Loss 4.8962 | Time 10.01
Epoch 01 | Step 045/090 | Loss 4.8296 | Time 12.28
Epoch 01 | Step 054/090 | Loss 4.6085 | Time 14.54
Epoch 01 | Step 063/090 | Loss 4.4226 | Time 16.80
Epoch 01 | Step 072/090 | Loss 4.2765 | Time 19.06
Epoch 01 | Step 081/090 | Loss 4.1730 | Time 21.32
Epoch 01 | Step 090/090 | Loss 4.0119 | Time 23.59
Start evaluating!
Current dev pearson is -0.1619, best pearson is 0.0000
Time costed : 24.774s 

—————————————————————— Epoch 2 ——————————————————————
Epoch 02 | Step 009/090 | Loss 2.1715 | Time 2.21
Epoch 02 | Step 018/090 | Loss 2.1066 | Time 4.42
Epoch 02 | Step 027/090 | Loss 2.1255 | Time 6.62
Epoch 02 | Step 036/090 | Loss 2.0307 | Time 8.83
Epoch 02 | Step 045/090 | Loss 1.9825 | Time 11.04
Epoch 02 | Step 054/090 | Loss 1.9421 | Time 13.25
Epoch 02 | Step 063/090 | Loss 1.8806 | 

In [21]:
hidden_size

768

In [22]:
dev_data.head(2)

,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,GEO,ENT,TIME,NAR,Overall,STYLE,TONE,text1,text2
0,en,en,1484189203_1484121193,https://wsvn.com/news/local/broward/police-2-m...,https://wsvn.com/news/local/no-swim-advisory-l...,https://web.archive.org/web/https://wsvn.com/n...,https://web.archive.org/web/https://wsvn.com/n...,1.5,4.0,2.0,4.0,3.5,1.0,1.5,"DAVIE, FLA. (WSVN) - Police need help catching...","DEERFIELD BEACH, FLA. (WSVN) - A no-swim advis..."
1,en,en,1484011097_1484011106,https://www.zdnet.com/article/autoclerk-databa...,https://securityboulevard.com/2019/10/best-wes...,https://web.archive.org/web/https://www.zdnet....,https://web.archive.org/web/https://securitybo...,1.0,2.0,1.0,1.0,1.0,3.5,2.5,Most Popular An open database exposing records...,The Home of the Security Bloggers Network Home...


In [23]:
# Delete columns that contain NaN sentences.
processed_test_data = dev_data[dev_data['text1'].notna()]
processed_test_data = processed_test_data[dev_data['text2'].notna()]

print("After removing NA text columns, we lose {0} rows.".format(dev_data.shape[0] - processed_test_data.shape[0]))

After removing NA text columns, we lose 0 rows.


In [24]:
processed_test_data['Geography'] = processed_test_data['GEO']
del processed_test_data['GEO']
processed_test_data['Entities'] = processed_test_data.pop('ENT')
processed_test_data['Time'] = processed_test_data.pop('TIME')
processed_test_data['Narrative'] = processed_test_data.pop('NAR')
processed_test_data['Overall'] = processed_test_data.pop('Overall')
processed_test_data['Style'] = processed_test_data.pop('STYLE')
processed_test_data['Tone'] = processed_test_data.pop('TONE')
processed_test_data['text1'] = processed_test_data.pop('text1')
processed_test_data['text2'] = processed_test_data.pop('text2')

In [25]:
test_data_loader = get_data_loader(processed_test_data, False)

In [26]:
len(processed_test_data)

4902

In [27]:
config = XLMRobertaConfig.from_pretrained("xlm-roberta-base")
hidden_size = config.hidden_size

pre_trained_model = XLMRobertaModel.from_pretrained("xlm-roberta-base")
model = MMRegressor(pre_trained_model, hidden_size)
model.load_state_dict(torch.load("/content/drive/MyDrive/CS263/Final Project/Models/XLM_Roberta_base.pth"), strict=False)
model.to(device)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


MMRegressor(
  (reg_model): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
          

In [28]:
hidden_size

768

## Evaluation

In [29]:
test_pred_overall, test_true_overall = predict(model, test_data_loader)
test_pearson_score = np.corrcoef(test_pred_overall, test_true_overall)[0][1]

print("Pearson score on test dataset is {:.3f}".format(test_pearson_score))


Pearson score on test dataset is 0.384


In [30]:
train_all = get_data_loader(processed_data, False)
train_pred_overall, train_true_overall = predict(model, train_all)
train_pearson_score = np.corrcoef(train_pred_overall, train_true_overall)[0][1]
print("Pearson score on entire train dataset is {:.3f}".format(train_pearson_score))

Pearson score on entire train dataset is 0.967


# TonyX

In [31]:
import pandas as pd
import time
import logging
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from transformers import XLMRobertaConfig, XLMRobertaModel, XLMRobertaTokenizer, AdamW, get_cosine_schedule_with_warmup

def set_seed(seed=56):
    random.seed(seed)
    os.environ['PYTHONHASHSEED']=str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

class MMRegressor(nn.Module):

    def __init__(self, model_path):
        
        super(MMRegressor, self).__init__()
        self.config = XLMRobertaConfig.from_pretrained(model_path)
        self.reg_model = XLMRobertaModel.from_pretrained(model_path)

        self.fc1 = nn.Linear(self.config.hidden_size, 512)
        self.fc2 = nn.Linear(512, 7)
        self.activation = nn.GELU()

    def forward(self, input_ids, attention_mask):

        output1 = self.reg_model(input_ids, attention_mask)[1]
        logits1 = self.fc2(self.activation(self.fc1(output1)))

        output2 = self.reg_model(input_ids, attention_mask)[1]
        logits2 = self.fc2(self.activation(self.fc1(output2)))
        
        return logits1, logits2

class Reg_FT_Configer():

    def __init__(self, params_dict: dict):
        
        super().__init__()
        
        self.learning_rate = params_dict['learning_rate']
        self.epoch =params_dict['epoch']
        self.gradient_acc = params_dict['gradient_acc']
        self.batch_size = params_dict['batch_size']
        self.max_len = params_dict['max_len']
        self.model_save_path = params_dict['model_save_path']
        self.warmup_rate = params_dict['warmup_rate']
        self.weight_decay = params_dict['weight_decay']
        self.model_pretrain_dir = params_dict['model_pretrain_dir']
        self.training_set_path = params_dict['training_set_path']
        self.testing_set_path = params_dict['testing_set_path']
        self.seed = params_dict['seed']

        # weights for the 7 sub-dimensions
        self.dims_weights = [params_dict['overall_weight'] if i == 4 else (1-params_dict['overall_weight'])/6 for i in range(7)]
        
        # weights for forward loss and adapted R-Drop loss
        self.losses_weights = {
            'forward_weight': (1-params_dict['rdrop_weight'])/2,
            'rdrop_weight': params_dict['rdrop_weight']
        }
        
        
class Reg_Trainer():

    def __init__(self, config: Reg_FT_Configer):

        super().__init__()

        self.config = config
        self.device = torch.device("cuda")
        self.tokenizer = XLMRobertaTokenizer.from_pretrained(self.config.model_pretrain_dir)

        set_seed(self.config.seed)


    def dataset(self, data_path):

        input_ids, attention_masks, labels = [], [], []

        for idx, row in pd.read_csv(data_path).iterrows():
            text1, text2 = row['text1'], row['text2']
            encode_dict = self.tokenizer.__call__(text1,text2,
                                                    max_length=self.config.max_len,
                                                    padding='max_length',
                                                    truncation=True,
                                                    add_special_tokens=True
                                                    )
            input_ids.append(encode_dict['input_ids'])
            attention_masks.append(encode_dict['attention_mask'])
            labels.append([float(x) for x in [row['Geography'],row['Entities'],row['Time'],row['Narrative'],row['Overall'],row['Style'],row['Tone']]])
        
        return torch.tensor(input_ids), torch.tensor(attention_masks), torch.tensor(labels)
    
    
    def data_loader(self, input_ids, attention_masks, labels):

        data = TensorDataset(input_ids, attention_masks, labels)
        loader = DataLoader(data, batch_size=self.config.batch_size, shuffle=True, drop_last=True)

        return loader

    def predict(self, model, data_loader):

        model.eval()
        test_pred, test_true = [], []
        with torch.no_grad():
            for idx, (ids, att, y) in enumerate(data_loader):
                y_pred = model(ids.to(self.device), att.to(self.device))
                y_pred = torch.squeeze(torch.add(torch.mul(y_pred[0], 0.5), torch.mul(y_pred[1], 0.5))).detach().cpu().numpy().tolist()
                y = y.squeeze().cpu().numpy().tolist()

                test_true.extend([x[4] for x in y])
                test_pred.extend([x[4] for x in y_pred])

            return test_true, test_pred


    def calculate_weighted_loss(self, y_pred, y, criterion):

        loss = 0.0
        for i in range(7):
            y_pred_i, y_i = y_pred[:, i], y[:, i]
            loss += criterion(y_pred_i, y_i) * self.config.dims_weights[i]
        return loss


    def train(self, model, train_loader, valid_loader, optimizer, schedule):

        best_pearson = 0.0
        criterion = nn.MSELoss()
        model.train()

        for i in range(self.config.epoch):
            start_time = time.time()
            train_loss_sum = 0.0
            
            logging.info(f"—————————————————————— Epoch {i+1} ——————————————————————")
            
            for idx, (ids, att, y) in enumerate(train_loader):

                ids, att, y = ids.to(self.device), att.to(self.device), y.to(self.device)
                y_pred1, y_pred2 = model(ids, att)
                y_pred1, y_pred2, y = torch.squeeze(y_pred1), torch.squeeze(y_pred2), torch.squeeze(y)

                loss1 = self.calculate_weighted_loss(y_pred1, y, criterion) * self.config.losses_weights['forward_weight']
                loss2 = self.calculate_weighted_loss(y_pred2, y, criterion) * self.config.losses_weights['forward_weight']
                loss_r = self.calculate_weighted_loss(y_pred1, y_pred2, criterion) * self.config.losses_weights['rdrop_weight']     
                loss = (loss1 + loss2 + loss_r) / self.config.gradient_acc

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                schedule.step()
                train_loss_sum += loss.item()

                if (idx+1) % (len(train_loader) // 10) == 0:
                    logging.info("Epoch {:02d} | Step {:03d}/{:03d} | Loss {:.4f} | Time {:.2f}".format(i+1, idx+1, len(train_loader), train_loss_sum/(idx+1), time.time()-start_time))
            

            logging.info("Start evaluating!")
            dev_true, dev_pred = self.predict(model, valid_loader)
            cur_pearson = np.corrcoef(dev_true, dev_pred)[0][1]
            
            if cur_pearson > best_pearson:
                best_pearson = cur_pearson
                torch.save(model.state_dict(), self.config.model_save_path)
            
            logging.info("Current dev pearson is {:.4f}, best pearson is {:.4f}".format(cur_pearson, best_pearson))
            logging.info("Time costed : {}s \n".format(round(time.time() - start_time, 3)))
    

    def run_finetune(self):

        train_loader = self.data_loader(*self.dataset(self.config.training_set_path))
        dev_loader = self.data_loader(*self.dataset(self.config.testing_set_path))

        model = MMRegressor(self.config.model_pretrain_dir).to(self.device)
        
        for param in model.parameters():
            param.requires_grad = True

        total_steps = len(train_loader) * self.config.epoch

        optimizer = AdamW(params=model.parameters(), 
                        lr=self.config.learning_rate, 
                        weight_decay=self.config.weight_decay)      
        schedule = get_cosine_schedule_with_warmup(optimizer=optimizer,
                                                    num_warmup_steps=self.config.warmup_rate*total_steps,
                                                    num_training_steps=total_steps)
        
        self.train(model, train_loader, dev_loader, optimizer, schedule)